<a href="https://colab.research.google.com/github/Alex210271/SD_WebUI_Copy/blob/main/SD_WebUI_by_dax_wrld_LITE2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 👇 **Запуск Stable Diffusion (~3мин)**


use_latest = False
from IPython.utils import capture
with capture.capture_output() as cap:
  %cd /content
del cap
# Install aria2 and lz4
import os
import time
from datetime import timedelta
from google.colab import drive
from subprocess import getoutput
from google.colab import runtime
import glob
import shutil
import re

try:
  start_colab
except:
  start_colab = int(time.time())

#Проверяем GPU

output = getoutput('nvidia-smi --query-gpu=gpu_name --format=csv')
if "name" in output:
  print('[32m 🤩 GPU-ускоритель подключен -', output[5:]) 
else:
  print("\033[91m 🙁 GPU-ускоритель НЕ подключен!\nПопробуйте переподключиться \nИли проверьте, возможно у вас закончились лимиты на использование GPU!")
  time.sleep(2)
  runtime.unassign()
print("\033[0m")




#@markdown  <font color="RoyalBlue">**::Сохранять изображения на Google-диск::**<br>
#@markdown  <font color="gray">Если чек-бокс установлен, изображения будут сохраняться на ваш G-диск в папку //WebUI/outputs/<br>
Save_Image_to_Gdrive= False #@param{type:"boolean"}






# @markdown <font color="RoyalBlue">**::Загрузка моделей::**<br>

# @markdown <font color="gray">- Original-Model-v1.5:<br><font color="gray">*Загрузка оригинальной модели v1.5. (Не занимает места на вашем G-диске)*<br>
# @markdown <font color="gray">- Google-Drive-AUTO:<br><font color="gray">*Автоматический поиск моделей на текущем Google-диске*<br>
# @markdown <font color="gray">- Folder-SDModels:<br><font color="gray">*Загрузка моделей из папки //SDModels*<br>
Models = "Original-Model-v1.5" #@param ["Original-Model-v1.5", "Google-Drive-AUTO", "Folder-SDModels"]

# @markdown <font color="gray">*- Или вставьте прямую ссылку на модель с сайта https://civitai.com*<br>
M_LINK = "" #@param {type:"string"}
## safetensors = False #@param {type:"boolean"}
# @markdown <font color="gray">Сохранить загруженную модель на Google-диске?<br>
Save_model_to_Gdrive= False #@param{type:"boolean"}


# @markdown <font color="RoyalBlue">**::Сервер::**<br>
# @markdown <font color="gray">Выбирайте любой, который у вас лучше работает<br>
Server = "BORE" #@param ["BORE", "GRADIO"]




vaes_dir = "/content/stable-diffusion-webui/models/VAE/"



if Save_Image_to_Gdrive or Save_model_to_Gdrive:
  if os.path.exists('/content/gdrive'):
    print("\033[32m")
    print("💾 Google-Диск подключен. Продолжаем.")
  else:   
    print("\033[35m")
    print("💾 Подключаем Google-Диск...", end='')
    if not os.path.exists('/content/gdrive'):
      with capture.capture_output() as cap:
        drive.mount('/content/gdrive')
      del cap
      print("\033[32m")
      print("💾 Google-Диск подключен.")

if Save_model_to_Gdrive:
  if not os.path.isdir(f"/content/gdrive/MyDrive/SDModels"):
    os.makedirs(f"/content/gdrive/MyDrive/SDModels")      
    

# ------------------------------------------------- #

print("\033[35m")
if not os.path.exists("/content/stable-diffusion-webui"):
  start_install = int(time.time())
  print("⌛ Установка SD... ", end='')
  with capture.capture_output() as cap:
    !apt install liblz4-tool aria2
    !rm -rf /usr/local/lib/python3.8/dist-packages/scipy /usr/local/lib/python3.8/dist-packages/scipy-1.7.3.dist-info /usr/local/lib/python3.8/dist-packages/scipy.libs
    !aria2c -c -x 16 -k 1M -s 16 -d "/content" -o "dep1001.tar.lz4" https://huggingface.co/daxwrld/SD-fast-repo-clone/resolve/main/dep1001.tar.lz4
    !aria2c -c -x 16 -k 1M -s 16 -d "/content" -o "repo1001.tar.lz4" https://huggingface.co/daxwrld/SD-fast-repo-clone/resolve/main/repo1001.tar.lz4
    !aria2c -c -x 16 -k 1M -s 16 -d "/content" -o "cache1001.tar.lz4" https://huggingface.co/daxwrld/SD-fast-repo-clone/resolve/main/cache1001.tar.lz4
    !tar -xI lz4 -f dep1001.tar.lz4 --directory=/usr/local/lib/python3.8/dist-packages/ #(manual dir)
    !tar -xI lz4 -f repo1001.tar.lz4 --directory=/ #/content/stable-diffusion-webui/ (auto dir)
    !tar -xI lz4 -f cache1001.tar.lz4 --directory=/ #/root/.cache/huggingface (auto dir)
    !rm -rf /content/dep1001.tar.lz4 /content/repo1001.tar.lz4 /content/cache1001.tar.lz4
    os.environ["SAFETENSORS_FAST_GPU"]='1'
    del cap

  if not 'T4' in getoutput('nvidia-smi --query-gpu=gpu_name --format=csv'):
    !pip uninstall -y xformers
  
  print("\033[32m")
  install_time = timedelta(seconds=time.time()-start_install)
  print("\r✅ Установка SD выполнена за:","%02d:%02d:%02d\n" % (install_time.seconds / 3600, (install_time.seconds / 60) % 60, install_time.seconds % 60), end='', flush=True)
else:
  print("\033[32m")
  print("✅ SD установлен. Продолжаем.")
  time_since_start = timedelta(seconds=time.time()-start_colab)
  print('\033[34m')
  print("⌚ Время использования колаба:","%02d:%02d:%02d" % (time_since_start.seconds / 3600, (time_since_start.seconds / 60) % 60, time_since_start.seconds % 60))

commit_hash = "" #11d432d92d63660c516540dcb48faac87669b4f0
if commit_hash:
  print('⌚ Активируем машину времени...', end="")
  with capture.capture_output() as cap:
    %cd /content/stable-diffusion-webui
    !git config --global user.email "you@example.com"
    !git config --global user.name "Your Name"
    !git stash
    !git reset --hard {commit_hash}
    !git stash pop
    del cap
  print('\r⌚ Машина времени активирована, SD возвращен к стабильной версии:', commit_hash, flush=True)

if Save_Image_to_Gdrive:
  config_dir="/content/stable-diffusion-webui/config.json"
  !sed -i 's@"outdir_txt2img_samples": "outputs/txt2img-images"@"outdir_txt2img_samples": "/content/gdrive/MyDrive/WebUI/outputs/txt2img-images"@' {config_dir}
  !sed -i 's@"outdir_img2img_samples": "outputs/img2img-images"@"outdir_img2img_samples": "/content/gdrive/MyDrive/WebUI/outputs/img2img-images"@' {config_dir}
  !sed -i 's@"outdir_extras_samples": "outputs/extras-images"@"outdir_extras_samples": "/content/gdrive/MyDrive/WebUI/outputs/extras-images"@' {config_dir}
  !sed -i 's@"outdir_txt2img_grids": "outputs/txt2img-grids"@"outdir_txt2img_grids": "/content/gdrive/MyDrive/WebUI/outputs/txt2img-grids"@' {config_dir}
  !sed -i 's@"outdir_img2img_grids": "outputs/img2img-grids"@"outdir_img2img_grids": "/content/gdrive/MyDrive/WebUI/outputs/img2img-grids"@' {config_dir}
  !sed -i 's@"outdir_save": "log/images"@"outdir_save": "/content/gdrive/MyDrive/WebUI/outputs/log/images"@' {config_dir}

# ------- Models selection (code design by noop_noob#0479)
#markdown ----
print("\033[35m")
print('⌛ Установка моделей...', end='')
var_model = ""

if M_LINK:
  if Save_model_to_Gdrive:
    path_model="/content/gdrive/MyDrive/SDModels"
    var_model = " --ckpt-dir /content/gdrive/MyDrive/SDModels"
  else:
    path_model="/content/stable-diffusion-webui/models/Stable-diffusion"
  print("\033[35m")
  print("⌛ Загружаем...", end='')
  with capture.capture_output() as cap:
    !apt install liblz4-tool aria2
    !aria2c -c -x 16 -k 1M -s 16 -d {path_model} {M_LINK}
  del cap
  print("\033[32m")
  print("✅ Модель загружена. Продолжаем.")

elif Models=="Original-Model-v1.5":
  if os.path.exists('/content/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt'):
    if os.path.getsize("/content/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt") > 1810671599:
      print("\033[32m")
      print("✅ Модель загружена (Original v1.5 pruned-emaonly). Продолжаем.")
  else:
    print("\033[35m")
    print("⌛ Модель не найдена. Загружаем...", end='')

    print("\033[0m")
    with capture.capture_output() as cap:
      !wget "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt" -O "/content/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt"
      del cap
    print("\033[32m")
    print("✅ Модель загружена (Original v1.5 pruned-emaonly). Продолжаем.")

elif Models=="Folder-SDModels":
  if os.path.exists('/content/gdrive'):
    if not Save_Image_to_Gdrive:
      print("\033[32m")
      print("💾 Google-Диск подключен. Продолжаем.")
  else:   
    print("\033[35m")
    print("💾 Подключаем Google-Диск...", end='')
    if not os.path.exists('/content/gdrive'):
      with capture.capture_output() as cap:
        drive.mount('/content/gdrive')
      del cap
      print("\033[32m")
      print("💾 Google-Диск подключен.")

  if not os.path.isdir(f"/content/gdrive/MyDrive/SDModels"):
    os.makedirs(f"/content/gdrive/MyDrive/SDModels")
    
  print("\033[32m")
  print("✅ Модели будут загружены из папки /SDModels.")
  var_model =" --ckpt-dir /content/gdrive/MyDrive/SDModels"


elif Models=='Google-Drive-AUTO':
  if os.path.exists('/content/gdrive'):
    if not Save_Image_to_Gdrive:
      print("\033[32m")
      print("💾 Google-Диск подключен. Продолжаем.")
  else:   
    print("\033[35m")
    print("💾 Подключаем Google-Диск...", end='')
    if not os.path.exists('/content/gdrive'):
      with capture.capture_output() as cap:
        drive.mount('/content/gdrive')
      del cap
      print("\033[32m")
      print("💾 Google-Диск подключен.")
    
  if not os.path.isdir(f"/content/gdrive/MyDrive/SDModels"):
    os.makedirs(f"/content/gdrive/MyDrive/SDModels")
  print("\033[35m")
  print("📦 Все модели, найденные на Google-диске, будут автоматически перенесены в папку /SDModels")
  print("⌛ Идёт поиск моделей...", end='')
  print("")
  for file in glob.glob('/content/gdrive/MyDrive/**/*.ckpt', recursive = True):
    if not file.find(f"/content/gdrive/MyDrive/SDModels") != -1:
      try:
        shutil.move(file, f"/content/gdrive/MyDrive/SDModels")
        print("\033[35m")
        print("⌛ Найдена модель:" + "\033[0m" + file + "\033[35m Переносим...", end='')
        print('')
      except:
        print("\033[91m ❌ Модель:" + "\033[0m" + file + "\033[91m уже есть в папке /SDModels. Пропускаем...")
    print("\033[32m ⤓ Загружена модель:"+"\033[0m" + file)   
  var_model = " --ckpt-dir /content/gdrive/MyDrive/SDModels"

with capture.capture_output() as cap:
  !wget "https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt" -O {vaes_dir}vae-ft-ema.ckpt
  !wget "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt" -O {vaes_dir}vae-ft-mse.ckpt
  del cap
  

  
var_server =""
with capture.capture_output() as cap:  
  if Server == "BORE":
    if not os.path.exists('/usr/bin/bore'):
      !curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
    !bore local 7860 --to bore.pub > bore.txt 2>&1 &
    time.sleep(2)
    !grep -o -m1 'bore.pub:[^ ]*' /content/bore.txt > boreport.txt
    boreport = getoutput('cat /content/boreport.txt')
    !sed -i '/self.server_name = /c\            self.server_name = "bore.pub"' /usr/local/lib/python3.8/dist-packages/gradio/blocks.py
    !sed -i '/self.server_port = /c\            self.server_port = {boreport[9:]}' /usr/local/lib/python3.8/dist-packages/gradio/blocks.py
    !sed -i '/    else "http/c\                else "http"' /usr/local/lib/python3.8/dist-packages/gradio/blocks.py
    !sed -i '/"PUBLIC_SHARE_TRUE":/c\    "PUBLIC_SHARE_TRUE": "[32mПодключено, используйте ссылку выше!⬆⬆⬆[0m",' /usr/local/lib/python3.8/dist-packages/gradio/strings.py
    !rm /content/bore.txt
    !rm /content/boreport.txt
del cap


if Server=="GRADIO":
  var_server = " --share --gradio-debug "

if Server=="NGROK":
  print()
  print("\033[35mВставьте в поле ниже ⬇⬇⬇ токен NGROK сочетанием клавиш 'Ctrl+V' и нажмите 'Enter'. Токен можно взять здесь: https://dashboard.ngrok.com/get-started/your-authtoken")
  print("\033[32m")
  ngrok_token = input ("Токен NGROK: ")
  ngrok_region = "eu"
  var_server = f" --ngrok {ngrok_token} --ngrok-region {ngrok_region}"


with capture.capture_output() as cap:
  %cd /content/stable-diffusion-webui
del cap

if use_latest:
  !git pull
  print("\033[0m")



print("\033[32m")
print("\033[1m")
print("🚀 Греем двигатели и запускаем Stable Diffusion 🚀")
print('\033[0m')

print("\033[35m")
if Server=="NGROK":
  print("👀 \033[35mОсталось совсем немного...  ⬇⬇⬇ Ожидайте ниже ссылку на вход в SD. Пример ссылки:" + "\033[0m ngrok connected to localhost:7860! URL: *-*-*-*.eu.ngrok.io", end='')
if Server=="GRADIO":
  print("👀 \033[35mОсталось совсем немного...  ⬇⬇⬇ Ожидайте ниже ссылку на вход в SD. Пример ссылки:" + "\033[0m Running on public URL: ****.gradio.live", end='')
if Server=="BORE":
  print("👀 \033[35mОсталось совсем немного...  ⬇⬇⬇ Ожидайте ниже ссылку на вход в SD", end='')

#Удаляем темную тему
with capture.capture_output() as cap:
  shutil.rmtree("/content/stable-diffusion-webui/extensions/sd-web-ui-quickcss", ignore_errors=True)
  shutil.rmtree("/content/stable-diffusion-webui/outputs", ignore_errors=True)
#Добавляем готовые стили
# !wget "https://huggingface.co/daxwrld/SD-fast-repo-clone/resolve/main/styles.csv" -O "/content/stable-diffusion-webui/styles.csv"

del cap




additional_arg= var_model + var_server + ' --vae-path ' + vaes_dir
print("\033[0m")
!COMMANDLINE_ARGS="--enable-insecure-extension-access --xformers --disable-safe-unpickle {additional_arg}" REQS_FILE="requirements.txt" python launch.py

time_since_start = timedelta(seconds=time.time()-start_colab)
print('\033[34m')
print("\n\n\033[34m ⌚ Время использования колаба:","%02d:%02d:%02d" % (time_since_start.seconds / 3600, (time_since_start.seconds / 60) % 60, time_since_start.seconds % 60))
print("\n\n")